In [51]:
import numpy
import scipy
import netCDF4
import matplotlib.pyplot as mp
import matplotlib.ticker
import matplotlib.colors
import scipy.stats
import pandas
import itertools
import datetime
import os

mp.rcParams.update({'mathtext.default': 'regular'})
#get_ipython().magic('matplotlib inline')
%matplotlib inline

In [31]:
PRECT_nlat = 26
PRECT_nlon = 25

latlon_indices = list(itertools.product(range(PRECT_nlat), range(PRECT_nlon)))

PRECT_lat = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lat.npy')
PRECT_lon = numpy.load('/Users/baird/Dropbox/_analysis/attribution_2017/NEW_CALCULATIONS/npy_files/PRECT_lon.npy')

In [3]:
threshold = 0.1

year_start_pic = 402 #time_subsets[chunk,0]
year_end_pic = 2200 #time_subsets[chunk,1]

year_start = 1920
year_end = 2100

lo_perc = 20
hi_perc = 80

In [33]:
ensemble_members = numpy.hstack((numpy.arange(1,36), numpy.arange(101,106)))
ensemble_names = ['{:03d}'.format(i) for i in ensemble_members]

In [9]:
working_dir = '/Users/baird/Dropbox/_data_original/NCAR_LENS/daily/PRECT/calculated_npy_files/whole_domain/'

In [14]:
# ======================================================================
# Open preindustrial control info (dict_pic)
# create season strings for PIC
years_pic = numpy.arange(year_start_pic, year_end_pic+1, 1).astype(numpy.int)
half_years_pic = numpy.arange(year_start_pic+0.75, year_end_pic, 1)

season_strings_pic = [str(years_pic[i])+'-'+str(years_pic[i+1]) for i in range(years_pic.size-1)]
member_strings_pic = ['{:03d}'.format(i) for i in range(1,36)]
n_seasons_pic=year_end_pic-year_start_pic
print(n_seasons_pic)

1798


In [15]:
all_seasonal_total_pic = numpy.zeros((n_seasons_pic, len(latlon_indices)))

In [16]:
for latlon_idx in range(len(latlon_indices)):

    if latlon_idx%10==0:
        print('latlon_idx', latlon_idx)

    filename = 'member_005_latidx_'+'{:02d}'.format(latlon_indices[latlon_idx][0])+'_lonidx_'+'{:02d}'.format(latlon_indices[latlon_idx][1])+'_years_'+'{:04d}'.format(year_start_pic)+'-'+'{:04d}'.format(year_end_pic)+'_threshold_'+str(threshold)+'mmday_'+region+'.npy'

    dict_pic = numpy.load(working_dir + filename).item()

    all_seasonal_total_pic[:,latlon_idx] = numpy.array([dict_pic[s]['seasonal_total'] for s in season_strings_pic])

latlon_idx 0
latlon_idx 10
latlon_idx 20
latlon_idx 30
latlon_idx 40
latlon_idx 50
latlon_idx 60
latlon_idx 70
latlon_idx 80
latlon_idx 90
latlon_idx 100
latlon_idx 110
latlon_idx 120
latlon_idx 130
latlon_idx 140
latlon_idx 150
latlon_idx 160
latlon_idx 170
latlon_idx 180
latlon_idx 190
latlon_idx 200
latlon_idx 210
latlon_idx 220
latlon_idx 230
latlon_idx 240
latlon_idx 250
latlon_idx 260
latlon_idx 270
latlon_idx 280
latlon_idx 290
latlon_idx 300
latlon_idx 310
latlon_idx 320
latlon_idx 330
latlon_idx 340
latlon_idx 350
latlon_idx 360
latlon_idx 370
latlon_idx 380
latlon_idx 390
latlon_idx 400
latlon_idx 410
latlon_idx 420
latlon_idx 430
latlon_idx 440
latlon_idx 450
latlon_idx 460
latlon_idx 470
latlon_idx 480
latlon_idx 490
latlon_idx 500
latlon_idx 510
latlon_idx 520
latlon_idx 530
latlon_idx 540
latlon_idx 550
latlon_idx 560
latlon_idx 570
latlon_idx 580
latlon_idx 590
latlon_idx 600
latlon_idx 610
latlon_idx 620
latlon_idx 630
latlon_idx 640


In [40]:
all_seasonal_total_pic_3d = all_seasonal_total_pic.reshape((1798,PRECT_nlat,PRECT_nlon))

In [44]:
print(all_seasonal_total_pic_3d.shape)

(1798, 26, 25)


# Save all seasonal data in a netcdf file

In [41]:
year_list = numpy.arange(1,1798+1)
time_datetime = [datetime.datetime(i,1,15) for i in year_list]
time_nc = netCDF4.date2num(time_datetime, units='days since 0001-01-01', calendar='standard')

In [53]:
# save hist clim
filename = 'seasonal_totals_PIC.nc'

if os.path.exists(filename):
    print('file exists')
    os.remove(filename)

ncfile = netCDF4.Dataset(filename, 'w', format='NETCDF4')

time_dim = ncfile.createDimension('time', None)
time_var = ncfile.createVariable('time', 'f4', ('time',))
time_var[:] = time_nc
time_var.units = 'days since 0001-01-01'

lat_dim = ncfile.createDimension('lat', PRECT_nlat)
lat_var = ncfile.createVariable('lat', 'f4', ('lat',))
lat_var[:] = PRECT_lat
lat_var.units = 'degrees North'

lon_dim = ncfile.createDimension('lon', PRECT_nlon)
lon_var = ncfile.createVariable('lon', 'f4', ('lon',))
lon_var[:] = PRECT_lon
lon_var.units = 'degrees East'

data_var = ncfile.createVariable('seasonal_total', 'f4', ('time','lat','lon',))
data_var[:] = all_seasonal_total_pic_3d[:,:,:]
data_var.units = 'Seasonal total precipitation (mm), with days <= 0.1 mm omitted to avoid drizzle effect'

ncfile.close()

file exists


# Now do same for hist + RCP8.5

In [69]:
n_ens = len(ensemble_names)
all_seasonal_total_hist_rcp = numpy.zeros((180, n_ens, len(latlon_indices)))

In [72]:
for ens_idx in range(len(ensemble_names)):
    print(ensemble_names[ens_idx])
    for latlon_idx in range(len(latlon_indices)):
        #if latlon_idx%10==0:
        #    print('latlon_idx', latlon_idx)
        ensemble_member=ensemble_names[ens_idx]
        filename = 'member_'+ensemble_member+'_latidx_'+'{:02d}'.format(latlon_indices[latlon_idx][0])+'_lonidx_'+'{:02d}'.format(latlon_indices[latlon_idx][1])+'_years_'+'{:04d}'.format(year_start)+'-'+'{:04d}'.format(year_end)+'_threshold_'+str(threshold)+'mmday_'+region+'.npy'
        #dict_list_hist_rcp.append(numpy.load(working_dir + filename).item())
        dict_hist_rcp = numpy.load(working_dir + filename).item()
        seasonal_totals = [dict_hist_rcp[i]['seasonal_total'] for i in dict_hist_rcp.keys()]
        all_seasonal_total_hist_rcp[:,ens_idx,latlon_idx] = seasonal_totals

001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
101
102
103
104
105


In [73]:
all_seasonal_total_hist_rcp_3d = all_seasonal_total_hist_rcp.reshape((180,n_ens,PRECT_nlat,PRECT_nlon))

In [74]:
print(all_seasonal_total_hist_rcp_3d.shape)

(180, 40, 26, 25)


In [75]:
year_list = numpy.arange(1921,2101)
time_datetime = [datetime.datetime(i,1,15) for i in year_list]
time_nc = netCDF4.date2num(time_datetime, units='days since 1920-01-01', calendar='standard')

In [77]:
# save hist clim
filename = 'seasonal_totals_hist_rcp.nc'

if os.path.exists(filename):
    print('file exists')
    os.remove(filename)

ncfile = netCDF4.Dataset(filename, 'w', format='NETCDF4')

time_dim = ncfile.createDimension('time', None)
time_var = ncfile.createVariable('time', 'f4', ('time',))
time_var[:] = time_nc
time_var.units = 'days since 1920-01-01'

lat_dim = ncfile.createDimension('lat', PRECT_nlat)
lat_var = ncfile.createVariable('lat', 'f4', ('lat',))
lat_var[:] = PRECT_lat
lat_var.units = 'degrees North'

lon_dim = ncfile.createDimension('lon', PRECT_nlon)
lon_var = ncfile.createVariable('lon', 'f4', ('lon',))
lon_var[:] = PRECT_lon
lon_var.units = 'degrees East'

ens_dim = ncfile.createDimension('ensemble', len(ensemble_names))
ens_var = ncfile.createVariable('ensemble', 'f4', ('ensemble',))
ens_var[:] = ensemble_names
ens_var.units = 'NCAR LENS ensemble member'

data_var = ncfile.createVariable('seasonal_total', 'f4', ('time','ensemble','lat','lon',))
data_var[:] = all_seasonal_total_hist_rcp_3d
data_var.units = 'Seasonal total precipitation (mm), with days <= 0.1 mm omitted to avoid drizzle effect'

ncfile.close()

file exists
